In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import netCDF4 as nc
from scipy.integrate import solve_ivp
import math
import matplotlib.animation as animation

In [7]:
def fill_nan_values(data):
    name = data.name
    index = data.index
    data_numpy = data.to_numpy()
    sorrounding_average = (data_numpy[2:] + data_numpy[:-2])/2
    for it, point in enumerate(data_numpy):
        if (math.isnan(point)):
            data_numpy[it] = sorrounding_average[it-1]
    data_new = pd.DataFrame(data=data_numpy, columns=[name], index=index)
    return data_new


def get_yearly_avg_lat_data(filename='Land_and_Ocean_LatLong1.nc', T_0='1900', T_star='2000'):
    """
    Get the yearly avg latitude data for temperature from the given filename, saves T_0 and T_star,
    which are files that will be used in both the budyko model analysis and the optimal control
    problem on the budyko-sellers model
    
    This code only looks at the top half of the globe to fit with the assumption that the earth temperature
    profile is symmetric. This is obviously not true, but for the massive system that is the earth, it is
    a reasonable assumption.
    
    Data is obtained from http://berkeleyearth.org/data/
    """
    print("Getting data from file...")
    data = nc.Dataset(filename)
    #get the latitude values
    lat_vals = np.array(data['latitude'][:])
    #get the yearly values
    time_vals = np.array(data['time'][:])
    time_ints = time_vals.astype(int)
    temp_array = np.array(data['temperature'][:])
    mask_semi_sphere = lat_vals > 0
    lat_vals_top = lat_vals[mask_semi_sphere]
    temp_array_top = temp_array[:,mask_semi_sphere,:]
    normal = np.array(data['climatology'][:])
    normal_top = normal[:,mask_semi_sphere,:]
    #get the year average
    print("Averaging over the entire year...")
    unique_years = np.unique(time_ints)
    m,n,o = np.shape(temp_array_top)
    temp_array_new = np.zeros((len(unique_years), n, o))
    #create the groupby and average over the years
    for it, year in enumerate(unique_years):
        mask_year = time_ints == year
        new_year = temp_array_top[mask_year,:,:]+normal_top
        mean_year = np.mean(new_year, axis=0)
        temp_array_new[it,:,:] = mean_year
    #now average the temp array over the latitudinal values
    print("Saving Data...")
    temp_array_final = np.mean(temp_array_new,axis=2)
    #create the dataframe
    lat_vals_top = np.sin(np.deg2rad(lat_vals_top))
    all_time_data_frame = pd.DataFrame(data=temp_array_final.T,index=lat_vals_top,columns=unique_years)
    #save the results to data frames using the given method parameters
    temp_T_star = fill_nan_values(all_time_data_frame[int(T_star)])
    temp_T_0 = fill_nan_values(all_time_data_frame[int(T_0)])
    #save the dataframes
    temp_T_star.to_csv('{}_mean_temp.csv'.format(T_star))
    temp_T_0.to_csv('{}_mean_temp.csv'.format(T_0))
    all_time_data_frame.to_csv('1850-2020_mean_temp.csv')
    print("Complete!")

In [8]:
get_yearly_avg_lat_data()

Getting data from file...


<ipython-input-7-ece6fdbb6f35>:32: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  temp_array = np.array(data['temperature'][:])
<ipython-input-7-ece6fdbb6f35>:32: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  temp_array = np.array(data['temperature'][:])
<ipython-input-7-ece6fdbb6f35>:36: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  normal = np.array(data['climatology'][:])
<ipython-input-7-ece6fdbb6f35>:36: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  normal = np.array(data['climatology'][:])


Averaging over the entire year...
Saving Data...
Complete!
